In [ ]:
!pip install langchain openai python-dotenv

In [ ]:
import os
from google.colab import userdata

# Securely store your API key in Colab Secrets (Tools → Secrets)
os.environ["OPENAI_API_KEY"] = "sk....."

In [4]:
# %%
from langchain.prompts import PromptTemplate

# Example 1: Simple Q&A Prompt
template = """
You are a helpful assistant. Answer the following question in 1-2 sentences.
Question: {question}
Answer:
"""

prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

# Test the prompt
question = "What is quantum computing?"
print(prompt.format(question=question))


You are a helpful assistant. Answer the following question in 1-2 sentences.
Question: What is quantum computing?
Answer:



In [5]:
# %%
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# Define examples
examples = [
    {
        "input": "Explain machine learning to a 5-year-old.",
        "output": "Machine learning is like teaching a robot to learn from examples, just like you learn to spot a cat by seeing lots of cat pictures!"
    },
    {
        "input": "What is photosynthesis?",
        "output": "Photosynthesis is how plants eat sunlight to make food. They use green stuff called chlorophyll to do this magic!"
    }
]

# Define example template
example_template = """
Input: {input}
Output: {output}
"""

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template=example_template
)

# Create few-shot prompt
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Input: {input}\nOutput:",  # Final instruction
    input_variables=["input"],
    example_separator="\n---\n"  # Separate examples with a divider
)

# Test with a new input
user_input = "Explain gravity to a 5-year-old."
print(few_shot_prompt.format(input=user_input))


Input: Explain machine learning to a 5-year-old.
Output: Machine learning is like teaching a robot to learn from examples, just like you learn to spot a cat by seeing lots of cat pictures!

---

Input: What is photosynthesis?
Output: Photosynthesis is how plants eat sunlight to make food. They use green stuff called chlorophyll to do this magic!

---
Input: Explain gravity to a 5-year-old.
Output:


In [7]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.14
    Uninstalling langchain-0.3.14:
      Successfully uninstalled langchain-0.3.14


In [9]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.5 MB/s eta 0:00:00


In [ ]:
# %%
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# Create example selector
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,  # Use the same examples as above
    embeddings=OpenAIEmbeddings(),
    vectorstore_cls=FAISS,
    k=1  # Number of examples to return
)

# Create a dynamic few-shot prompt
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,  # Use selector instead of fixed examples
    example_prompt=example_prompt,
    suffix="Input: {input}\nOutput:",
    input_variables=["input"],
    example_separator="\n---\n"
)

# Test with a similar input (will fetch the most relevant example)
user_input = "Describe electricity to a child."
print(dynamic_prompt.format(input=user_input))

In [ ]:
# %%
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# Create example selector
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,  # Use the same examples as above
    embeddings=OpenAIEmbeddings(),
    vectorstore_cls=FAISS,
    k=1  # Number of examples to return
)

# Create a dynamic few-shot prompt
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,  # Use selector instead of fixed examples
    example_prompt=example_prompt,
    suffix="Input: {input}\nOutput:",
    input_variables=["input"],
    example_separator="\n---\n"
)

# Test with a similar input (will fetch the most relevant example)
user_input = "Describe electricity to a child."
print(dynamic_prompt.format(input=user_input))

In [ ]:
# %%
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

# Define response schema
response_schemas = [
    ResponseSchema(name="answer", type="string", description="The answer to the question"),
    ResponseSchema(name="confidence", type="integer", description="Confidence score from 1-10")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Create a prompt with format instructions
template = """
Answer the question as truthfully as possible.
{format_instructions}
Question: {question}
Answer:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["question"],
    partial_variables={
        "format_instructions": output_parser.get_format_instructions()
    }
)

# Test with an LLM
llm = OpenAI(temperature=0)
user_input = "What is the capital of France?"
formatted_prompt = prompt.format(question=user_input)
response = llm(formatted_prompt)
parsed_response = output_parser.parse(response)

print(f"Raw response: {response}")
print(f"Parsed response: {parsed_response}")